In [1331]:
import numpy as np
import nltk
import pickle
import pandas as pd
import sys
import keras

import os
# mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-7.1.0-posix-seh-rt_v5-rev0\\mingw64\\bin'
# os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
    
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [1332]:
# train_df = pd.read_csv("btc_train.csv")
# train_df = pd.read_csv("eth_bitfinex.csv")
train_df = pd.read_csv("eth_binance.csv")
train_df = train_df.dropna()

In [1333]:
y_raw = train_df["ohlcv_close"].values
x_raw = train_df.drop(['time', 'last_price', 'ohlcv_close'], axis=1).values
# x_raw = scaler.fit_transform(x_raw)

log_count = len(y_raw)
feats = len(x_raw[0])
shift = 100
shift_y = 100
x_train = np.zeros((log_count-shift-shift_y,shift,feats))
y_train = np.zeros(log_count-shift-shift_y)
y_train_trend = np.zeros(log_count-shift-shift_y)

In [1334]:
for i in range(log_count-shift_y-shift):
    x_train[i] = x_raw[i:i+shift]
    y_train[i] = y_raw[i+shift+shift_y]
    diff = y_raw[i+shift+shift_y]-y_raw[i+shift]
    if diff/y_raw[i+shift] > 0.01 :
        y_train_trend[i] = 0
        
    elif diff/y_raw[i+shift] < -0.01:
        y_train_trend[i] = 1
        
    else:
        y_train_trend[i] = 2

In [1335]:
for i in range(3):
    print("class{} = {}".format(i,sum(y_train_trend==i)))

class0 = 221
class1 = 242
class2 = 10713


In [1336]:
cla_0 = [idx for idx,i in enumerate(y_train_trend) if i == 0]
cla_1 = [idx for idx,i in enumerate(y_train_trend) if i == 1]
cla_2 = [idx for idx,i in enumerate(y_train_trend) if i == 2]

In [1337]:
from sklearn.utils import resample
cla_2_idx = resample(cla_2, n_samples=1000, random_state=0)

In [1338]:
x_train = np.vstack((x_train[cla_0],x_train[cla_1],x_train[cla_2_idx]))
y_train_trend = np.hstack((y_train_trend[cla_0],y_train_trend[cla_1],y_train_trend[cla_2_idx]))

In [1339]:
x_train_xgb = x_train.reshape(-1, x_train.shape[1]*x_train.shape[2])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_trend, y_test_trend = train_test_split(x_train_xgb, y_train_trend, test_size=0.2, random_state=42)

# split = int(x_train_xgb.shape[0]*0.8)
# x_test = x_train_xgb[split:]
# y_test = y_train[split:]
# y_test_trend = y_train_trend[split:]
# x_train = x_train_xgb[:split]
# y_train = y_train[:split]
# y_train_trend = y_train_trend[:split]

In [1340]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
rf = RandomForestClassifier(n_estimators=500, max_depth=12, random_state=0, n_jobs=4)
rf.fit(x_train, y_train_trend)
# RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#            max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
print(sum(rf.feature_importances_>0.001))
print(confusion_matrix(rf.predict(x_test), y_test_trend))
print(f1_score(rf.predict(x_test), y_test_trend, average='macro')) 

323
[[ 36   0   3]
 [  0  52   0]
 [  2   0 200]]
0.9742397520175298


In [ ]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

params = {"objective": "reg:linear", "booster":"gbtree", 'max_depth':'2', 'eta':'0.0025', 'subsample':'0.7', 'eval_metric':'mae' , 'verbose':1}
params['nthread'] = 3  
evallist  = [(dtest,'eval')]
num_round = 1500
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

In [ ]:
a = x_train[:,rf.feature_importances_>0.00]
b = x_test[:,rf.feature_importances_>0.000]
dtrain = xgb.DMatrix(a, label=y_train)
dtest = xgb.DMatrix(b, label=y_test)

params = {"objective": "reg:linear", "booster":"gbtree", 'max_depth':'2', 'eta':'0.003', 'subsample':'0.7', 'eval_metric':'mae' , 'verbose':1}
params['nthread'] = 3  
evallist  = [(dtrain,'eval'),(dtest,'eval')]
num_round = 1440
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

In [1345]:
dtrain = xgb.DMatrix(x_train[:,rf.feature_importances_>0.0005], label=y_train_trend)
dtest = xgb.DMatrix(x_test[:,rf.feature_importances_>0.0005], label=y_test_trend)

params = {"objective": "multi:softmax", "booster":"gbtree", 'max_depth':'2', 'eta':'0.001', 'subsample':'0.7', 'eval_metric':'mlogloss' , 'verbose':0, 'num_class':3}
params['nthread'] = 4  
evallist  = [(dtest,'eval')]
num_round = 5000
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

[0]	eval-mlogloss:1.09776
[1]	eval-mlogloss:1.09694
[2]	eval-mlogloss:1.09611
[3]	eval-mlogloss:1.09529
[4]	eval-mlogloss:1.09447
[5]	eval-mlogloss:1.09361
[6]	eval-mlogloss:1.09278
[7]	eval-mlogloss:1.092
[8]	eval-mlogloss:1.09123
[9]	eval-mlogloss:1.09039
[10]	eval-mlogloss:1.08959
[11]	eval-mlogloss:1.08875
[12]	eval-mlogloss:1.08793
[13]	eval-mlogloss:1.08714
[14]	eval-mlogloss:1.08632
[15]	eval-mlogloss:1.08554
[16]	eval-mlogloss:1.08473
[17]	eval-mlogloss:1.08393
[18]	eval-mlogloss:1.08306
[19]	eval-mlogloss:1.08228
[20]	eval-mlogloss:1.08147
[21]	eval-mlogloss:1.08066
[22]	eval-mlogloss:1.07979
[23]	eval-mlogloss:1.07901
[24]	eval-mlogloss:1.07822
[25]	eval-mlogloss:1.07742
[26]	eval-mlogloss:1.07663
[27]	eval-mlogloss:1.0758
[28]	eval-mlogloss:1.07497
[29]	eval-mlogloss:1.0742
[30]	eval-mlogloss:1.07338
[31]	eval-mlogloss:1.07251
[32]	eval-mlogloss:1.07173
[33]	eval-mlogloss:1.07089
[34]	eval-mlogloss:1.07008
[35]	eval-mlogloss:1.06925
[36]	eval-mlogloss:1.06847
[37]	eval-mlogl

[292]	eval-mlogloss:0.896326
[293]	eval-mlogloss:0.895763
[294]	eval-mlogloss:0.895179
[295]	eval-mlogloss:0.89464
[296]	eval-mlogloss:0.894067
[297]	eval-mlogloss:0.893538
[298]	eval-mlogloss:0.892998
[299]	eval-mlogloss:0.892478
[300]	eval-mlogloss:0.891903
[301]	eval-mlogloss:0.891343
[302]	eval-mlogloss:0.890778
[303]	eval-mlogloss:0.890209
[304]	eval-mlogloss:0.889658
[305]	eval-mlogloss:0.889098
[306]	eval-mlogloss:0.888539
[307]	eval-mlogloss:0.887962
[308]	eval-mlogloss:0.887408
[309]	eval-mlogloss:0.886853
[310]	eval-mlogloss:0.8863
[311]	eval-mlogloss:0.885738
[312]	eval-mlogloss:0.88519
[313]	eval-mlogloss:0.884642
[314]	eval-mlogloss:0.884076
[315]	eval-mlogloss:0.883517
[316]	eval-mlogloss:0.882966
[317]	eval-mlogloss:0.882426
[318]	eval-mlogloss:0.881838
[319]	eval-mlogloss:0.881286
[320]	eval-mlogloss:0.880766
[321]	eval-mlogloss:0.880219
[322]	eval-mlogloss:0.879651
[323]	eval-mlogloss:0.879061
[324]	eval-mlogloss:0.878506
[325]	eval-mlogloss:0.877926
[326]	eval-mloglos

[576]	eval-mlogloss:0.757639
[577]	eval-mlogloss:0.757224
[578]	eval-mlogloss:0.75679
[579]	eval-mlogloss:0.75639
[580]	eval-mlogloss:0.75595
[581]	eval-mlogloss:0.755537
[582]	eval-mlogloss:0.755116
[583]	eval-mlogloss:0.754732
[584]	eval-mlogloss:0.754349
[585]	eval-mlogloss:0.753946
[586]	eval-mlogloss:0.753505
[587]	eval-mlogloss:0.753123
[588]	eval-mlogloss:0.752707
[589]	eval-mlogloss:0.752283
[590]	eval-mlogloss:0.751893
[591]	eval-mlogloss:0.751479
[592]	eval-mlogloss:0.751064
[593]	eval-mlogloss:0.750622
[594]	eval-mlogloss:0.750212
[595]	eval-mlogloss:0.749808
[596]	eval-mlogloss:0.749403
[597]	eval-mlogloss:0.749005
[598]	eval-mlogloss:0.748621
[599]	eval-mlogloss:0.748176
[600]	eval-mlogloss:0.747767
[601]	eval-mlogloss:0.74734
[602]	eval-mlogloss:0.746913
[603]	eval-mlogloss:0.746514
[604]	eval-mlogloss:0.746081
[605]	eval-mlogloss:0.745632
[606]	eval-mlogloss:0.74521
[607]	eval-mlogloss:0.74484
[608]	eval-mlogloss:0.744435
[609]	eval-mlogloss:0.74401
[610]	eval-mlogloss:0

[860]	eval-mlogloss:0.653279
[861]	eval-mlogloss:0.652988
[862]	eval-mlogloss:0.652661
[863]	eval-mlogloss:0.652351
[864]	eval-mlogloss:0.652019
[865]	eval-mlogloss:0.651704
[866]	eval-mlogloss:0.65138
[867]	eval-mlogloss:0.651058
[868]	eval-mlogloss:0.65074
[869]	eval-mlogloss:0.650401
[870]	eval-mlogloss:0.650089
[871]	eval-mlogloss:0.649764
[872]	eval-mlogloss:0.64943
[873]	eval-mlogloss:0.649132
[874]	eval-mlogloss:0.648803
[875]	eval-mlogloss:0.648486
[876]	eval-mlogloss:0.648202
[877]	eval-mlogloss:0.647893
[878]	eval-mlogloss:0.647587
[879]	eval-mlogloss:0.647294
[880]	eval-mlogloss:0.646962
[881]	eval-mlogloss:0.646645
[882]	eval-mlogloss:0.646333
[883]	eval-mlogloss:0.646007
[884]	eval-mlogloss:0.645715
[885]	eval-mlogloss:0.64538
[886]	eval-mlogloss:0.645086
[887]	eval-mlogloss:0.644777
[888]	eval-mlogloss:0.644463
[889]	eval-mlogloss:0.644154
[890]	eval-mlogloss:0.64383
[891]	eval-mlogloss:0.643535
[892]	eval-mlogloss:0.64321
[893]	eval-mlogloss:0.642904
[894]	eval-mlogloss:

[1139]	eval-mlogloss:0.573529
[1140]	eval-mlogloss:0.573285
[1141]	eval-mlogloss:0.573002
[1142]	eval-mlogloss:0.572751
[1143]	eval-mlogloss:0.57251
[1144]	eval-mlogloss:0.57226
[1145]	eval-mlogloss:0.571981
[1146]	eval-mlogloss:0.571747
[1147]	eval-mlogloss:0.571475
[1148]	eval-mlogloss:0.571206
[1149]	eval-mlogloss:0.570947
[1150]	eval-mlogloss:0.570709
[1151]	eval-mlogloss:0.570468
[1152]	eval-mlogloss:0.570225
[1153]	eval-mlogloss:0.569997
[1154]	eval-mlogloss:0.569727
[1155]	eval-mlogloss:0.569462
[1156]	eval-mlogloss:0.569225
[1157]	eval-mlogloss:0.568986
[1158]	eval-mlogloss:0.568735
[1159]	eval-mlogloss:0.568494
[1160]	eval-mlogloss:0.568243
[1161]	eval-mlogloss:0.56797
[1162]	eval-mlogloss:0.567673
[1163]	eval-mlogloss:0.567414
[1164]	eval-mlogloss:0.567172
[1165]	eval-mlogloss:0.566905
[1166]	eval-mlogloss:0.566659
[1167]	eval-mlogloss:0.566385
[1168]	eval-mlogloss:0.566141
[1169]	eval-mlogloss:0.565905
[1170]	eval-mlogloss:0.565683
[1171]	eval-mlogloss:0.565425
[1172]	eval-m

KeyboardInterrupt: 

In [1342]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
print(confusion_matrix(gbm_1.predict(dtest), y_test_trend))
print(f1_score(gbm_1.predict(dtest), y_test_trend, average='macro')) 

[[ 22   0   8]
 [  0  50   3]
 [ 16   2 192]]
0.8430739527449398


In [1220]:
gbm_1.predict_proba(dtest)

AttributeError: 'Booster' object has no attribute 'predict_proba'